In [1]:
import os
import seaborn as sns
import pandas as pd
import numpy as np

os.chdir('../scripts')
import nomad_request

In [2]:
nomad_gfs = nomad_request.retrieve_nomad()

In [8]:
nomad_gfs.head()

<xarray.Dataset>
Dimensions:     (latitude: 5, longitude: 5)
Coordinates:
    time        datetime64[ns] 2022-02-20T18:00:00
    step        timedelta64[ns] 00:00:00
    surface     int64 0
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 89.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 1.0
    valid_time  datetime64[ns] 2022-02-20T18:00:00
Data variables:
    sp          (latitude, longitude) float32 ...
    orog        (latitude, longitude) float32 ...
    prate       (latitude, longitude) float32 ...
    veg         (latitude, longitude) float32 ...
    slt         (latitude, longitude) float32 ...
    lftx        (latitude, longitude) float32 ...
    cape        (latitude, longitude) float32 ...
    cin         (latitude, longitude) float32 ...
    4lftx       (latitude, longitude) float32 ...
    ist         (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-02-20T20:24:45 GRIB to CDM+CF via cfgrib-0....

Using the `.sel` method of the xarray dataset, we can extract a specific slice corresponding to a particular location. For example, if we have a survey location at 21.31, -157.64 (which is the location of the Makapu'u lighthouse on the east side of Oahu), we can get the four closest GFS forecast locations using the following approach. Note that since GFS' longitude measures are bounded (0,360) rather than (-180, 180), we need to add 180 to find the proper longitude for GFS subsetting. 

In [4]:
sliced = nomad_gfs.sel({'latitude':[21.25, 21.50], 'longitude': [-157.50 + 180, 157.75 + 180]})


In [5]:
sliced[['sp','prate']].to_array().to_numpy()

array([[[9.5167180e+04, 1.0198318e+05],
        [9.5643984e+04, 1.0201358e+05]],

       [[0.0000000e+00, 2.4000001e-06],
        [0.0000000e+00, 8.0000000e-06]]], dtype=float32)

This approach has been incorporated into the function `retrieve_closest_points` in `nomad_request.py`:

In [12]:
sliced = nomad_request.retrieve_closest_points(
    nomad_gfs, (21.31, -157.64), var_list = ['sp', 'orog'], out_type = 'xarray'
)

In [13]:
sliced

<xarray.DataArray (variable: 2, latitude: 2, longitude: 2)>
array([[[95479.18   , 95167.18   ],
        [95803.984  , 95643.984  ]],

       [[  575.2594 ,   602.77936],
        [  546.6194 ,   560.3794 ]]], dtype=float32)
Coordinates:
    time        datetime64[ns] 2022-02-20T18:00:00
    step        timedelta64[ns] 00:00:00
    surface     int64 0
  * latitude    (latitude) float64 21.25 21.5
  * longitude   (longitude) float64 22.25 22.5
    valid_time  datetime64[ns] 2022-02-20T18:00:00
  * variable    (variable) <U4 'sp' 'orog'
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-02-20T20:24:45 GRIB to CDM+CF via cfgrib-0....